## Watermark photo attachments with Exif data

This notebook demonstrates how to watermark photo attachments with latitude and longitude from Exif GPS info using the Pillow Library.

To use this notebook, you need to update variables at the top of the script, including:

1. feature_layer_item_id to the item id of the feature layer to watermark attachments for and feature_layer_id to specify the specific layer
2. attachments_query to filter what features to watermark their attachments

This notebook assumes that:

- You are the owner of the webmap
- You are using Enterprise 10.7+ or ArcGIS Online

For more information on working with map areas see:

- ArcGIS Python API Layer Attachments - https://developers.arcgis.com/python/guide/using-attachments-with-feature-layers/
- Pillow Library - https://pillow.readthedocs.io/en/stable/index.html

In [ ]:
import os
import shutil
import tempfile
from arcgis.gis import GIS
from datetime import datetime as dt
from PIL import ExifTags
from PIL import Image
from PIL import ImageDraw
from PIL import ImageOps


# Using built-in user
# For information on different authentication schemes see
# https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/
# and for protecting your credentials see
# https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#protecting-your-credentials
gis = GIS("home")

# The item id of the feature layer to process attachments for
feature_layer_item_id = "7a28f3f872bd4b55bfd7965bf73a46b6"
# The id of the layer to use
feature_layer_id = 0
# Where clause to filter features and attachments
# See FeatureLayer query() for more options - https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featurelayer
attachments_query = "field_to_filter=2"


# Converts latitude and longitude from Exif GPS info in degrees, minutes and seconds to decimal degrees
def convert_dms_to_dd(gps_info) -> (float, float):

    if gps_info is None:
        return (0, 0)

    lat_degrees = gps_info[ExifTags.GPS.GPSLatitude][0]
    lat_minutes = gps_info[ExifTags.GPS.GPSLatitude][1]
    lat_seconds = gps_info[ExifTags.GPS.GPSLatitude][2]
    lon_degrees = gps_info[ExifTags.GPS.GPSLongitude][0]
    lon_minutes = gps_info[ExifTags.GPS.GPSLongitude][1]
    lon_seconds = gps_info[ExifTags.GPS.GPSLongitude][2]

    lat = float(lat_degrees + lat_minutes / 60 + lat_seconds / 3600)
    lon = float(lon_degrees + lon_minutes / 60 + lon_seconds / 3600)

    if gps_info[ExifTags.GPS.GPSLatitudeRef] == "S":
        lat = -lat
    if gps_info[ExifTags.GPS.GPSLongitudeRef] == "W":
        lon = -lon

    return (lat, lon)


# Watermarks an image with decimal degree latitude and longitude from Exif data
def add_dd_watermark(image_path):

    image = Image.open(image_path)
    image = ImageOps.exif_transpose(image)

    # Get the image's Exif data, convert the lat and long to decimal degree and format the string for the watermark
    exif = image.getexif()
    gps_ifd = exif.get_ifd(ExifTags.IFD.GPSInfo)
    (lat, lon) = convert_dms_to_dd(gps_ifd)
    watermark_text = f"Latitude: {lat:.4f} Longitude: {lon:.4f}"

    draw = ImageDraw.Draw(image)

    # Draw the watermark in the top left corner of the image in white
    # For more drawing options see: https://pillow.readthedocs.io/en/stable/reference/ImageDraw.html#PIL.ImageDraw.ImageDraw.text
    # See https://pillow.readthedocs.io/en/stable/reference/ImageFont.html for information on how to change the font, including font size
    text_xy_position = (10, 10)
    rgb_fill_color = (255, 255, 255)
    draw.text(text_xy_position, watermark_text, fill=rgb_fill_color)
    image.save(image_path)
    # Optional call to show the photo with the watermark
    image.show()


# Create temporary output directories
temporary_attachment_output = tempfile.TemporaryDirectory()
temporary_zip_output = tempfile.TemporaryDirectory()
temporary_zip_output_name = os.path.join(
    temporary_zip_output.name,
    "zipped_photos_{}".format(dt.now().strftime("%Y-%m-%d %H-%M-%S")),
)

# Query for features that match the where clause and download their attachments to a temporary output directory
layer_item = gis.content.get(feature_layer_item_id)
layer = layer_item.layers[feature_layer_id]
oid_list = layer.query(where=attachments_query, return_ids_only=True)["objectIds"]
attachments = layer.attachments.download(
    oid=oid_list, save_path=temporary_attachment_output.name
)

# For each of the attachments watermark the image
for attachment in attachments:
    add_dd_watermark(attachment)

# Create a zip archive with the watermarked images
output_zip = shutil.make_archive(
    temporary_zip_output_name, "zip", temporary_attachment_output.name
)

# Create a new Image Collection item with the archived watermarked images
item_props = {
    "type": "Image Collection",
    "typeKeywords": ["Image", "Image Collection", "Photo locations"],
    "description": "Images with watermarks",
    "title": "Images with watermarks",
    "tags": [],
    "snippet": "Images with watermarks",
}
new_item = gis.content.add(item_properties=item_props, data=output_zip)

# Clean up temporary files
os.remove(output_zip)
temporary_attachment_output.cleanup()
temporary_zip_output.cleanup()

display(new_item)
